# Installations

In [1]:
!pip install transformers datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 547.8/547.8 kB 7.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 40.8/40.8 MB 13.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 6.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.9/64.9 kB 4.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.1 MB/s eta 0:00:00
  Attempting uninstall: requests
    Found existing installation: requests 2.31.0
    Uninstalling requests-2.31.0:
      Successfully uninstalled requests-2.31.0
  Attempting uninstall: pyarrow
    Found existing installation: pyarrow 14.0.2
    Uninstalling pyarrow-14.0.2:
      Successfully uninstalled pyarrow-14.0.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf-cu12 24

In [2]:
import tensorflow as tf### models
import numpy as np### math computations
import matplotlib.pyplot as plt### plotting bar chart
import sklearn### machine learning library
import cv2## image processing
from sklearn.metrics import confusion_matrix, roc_curve### metrics
import seaborn as sns### visualizations
import datetime
import pathlib
import io
import os
import re
import string
import time
from numpy import random
import gensim.downloader as api
from PIL import Image
import tensorflow_datasets as tfds
import tensorflow_probability as tfp
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Layer
from tensorflow.keras.layers import Dense,Flatten,InputLayer,BatchNormalization,Dropout,Input,LayerNormalization
from tensorflow.keras.losses import BinaryCrossentropy,CategoricalCrossentropy, SparseCategoricalCrossentropy
from tensorflow.keras.metrics import Accuracy,TopKCategoricalAccuracy, CategoricalAccuracy, SparseCategoricalAccuracy
from tensorflow.keras.optimizers import Adam
from google.colab import drive
from google.colab import files
from datasets import load_dataset
from transformers import GPT2TokenizerFast,create_optimizer,DataCollatorForLanguageModeling,TFGPT2LMHeadModel

In [29]:
MAX_LENGTH=256
BATCH_SIZE=8

# Dataset Preparation

In [4]:
!pip install -q kaggle
!mkdir ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 /root/.kaggle/kaggle.json
!kaggle datasets download -d juicobowley/drake-lyrics
!unzip "/content/drake-lyrics.zip" -d "/content/dataset/"

cp: cannot stat 'kaggle.json': No such file or directory
chmod: cannot access '/root/.kaggle/kaggle.json': No such file or directory
Dataset URL: https://www.kaggle.com/datasets/juicobowley/drake-lyrics
License(s): other
100% 764k/764k [00:00<00:00, 879kB/s]
100% 764k/764k [00:00<00:00, 879kB/s]
Archive:  /content/drake-lyrics.zip
  inflating: /content/dataset/drake_data.csv  
  inflating: /content/dataset/drake_data.json  
  inflating: /content/dataset/drake_lyrics.txt  


In [5]:
filepath="/content/dataset/drake_data.csv"
dataset = load_dataset('csv', data_files=filepath)

Generating train split: 0 examples [00:00, ? examples/s]

In [6]:
dataset

DatasetDict({
    train: Dataset({
        features: ['album', 'lyrics_title', 'lyrics_url', 'lyrics', 'track_views'],
        num_rows: 290
    })
})

In [7]:
dataset['train'][184]

{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [8]:
model_id="gpt2-medium"
tokenizer = GPT2TokenizerFast.from_pretrained(model_id)

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/huggingface_hub/file_download.py:1132: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

In [9]:
dataset['train'][184]

{'album': 'Thank Me Later',
 'lyrics_title': 'Thank Me Later [Booklet] Lyrics',
 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated',
 'lyrics': None,
 'track_views': '6.2K'}

In [10]:
n_wasted=0

In [11]:
# for i in range(len(dataset['train'])):
#   try:
#     outputs = tokenizer(
#       dataset["train"][i]["lyrics"],
#       truncation=True,
#       max_length=256,
#       return_overflowing_tokens=True,
#       return_length=True,
#     )
#     print(i,outputs['length'])

#     for k in outputs['length']:
#       if k!=256:
#         n_wasted+=k
#   except:
#     print('----------------------->i',i)

In [12]:
print(n_wasted)

0


In [13]:
def preprocess_function(example):
  try:
    outputs = tokenizer(
        example["lyrics"],
        truncation=True,
        max_length=MAX_LENGTH,
        return_overflowing_tokens=True,
        return_length=True,
    )
    input_batch = []
    for length, input_ids in zip(outputs["length"], outputs["input_ids"]):
      if length==MAX_LENGTH:
        input_batch.append(input_ids)
        valid_input_ids=input_ids
    if len(input_batch)!=0:
      for i in range(BATCH_SIZE-len(input_batch)):
        input_batch.append(valid_input_ids)
  except:
    print(example)
    input_batch=[]
  return {"input_ids": input_batch}

In [14]:
tokenized_dataset=dataset.map(
    preprocess_function,remove_columns=dataset["train"].column_names
)

Map:   0%|          | 0/290 [00:00<?, ? examples/s]

{'album': 'Thank Me Later', 'lyrics_title': 'Thank Me Later [Booklet] Lyrics', 'lyrics_url': 'https://genius.com/Drake-thank-me-later-booklet-annotated', 'lyrics': None, 'track_views': '6.2K'}
{'album': 'Unreleased Songs', 'lyrics_title': 'Untitled DaBaby Collaboration* (Ft. DaBaby) Lyrics', 'lyrics_url': 'https://genius.com/Drake-untitled-dababy-collaboration-lyrics', 'lyrics': None, 'track_views': '(Unreleased)'}


In [15]:
tokenized_dataset

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 290
    })
})

In [16]:
def filter_out(example):
  if len(example['input_ids'])>=1:
    return example

In [17]:
tokenized_full_dataset=tokenized_dataset.filter(filter_out)
print(tokenized_full_dataset)

Filter:   0%|          | 0/290 [00:00<?, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['input_ids'],
        num_rows: 270
    })
})


In [18]:
max_batch_len=0

In [19]:
for i in range(270):
  if len(tokenized_full_dataset['train'][i]['input_ids'])>max_batch_len:
    max_batch_len=len(tokenized_full_dataset['train'][i]['input_ids'])
  #print(i,len(tokenized_full_dataset['train'][i]['input_ids']))

In [20]:
print(max_batch_len)

6


In [21]:
tokenizer.pad_token = tokenizer.eos_token
data_collator = DataCollatorForLanguageModeling(tokenizer, mlm=False, return_tensors="tf")

In [22]:
tf_train_dataset = tokenized_full_dataset["train"].to_tf_dataset(
    columns=["input_ids","attention_mask", "labels"],
    collate_fn=data_collator,
    shuffle=True,
    batch_size=1,
)

In [23]:
for i in tf_train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   198, 11633,    77],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6), dtype=int64, numpy=array([[1, 1, 1, 1, 1, 1]])>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 13414,   325, ...,   198, 11633,    77],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826],
        [  447,   247,    83, ...,     6,   340,   826]]])>}


In [24]:
def adjust_attention_mask(input):
  return {'input_ids':input['input_ids'],
          'attention_mask':tf.ones([1,BATCH_SIZE,MAX_LENGTH]),
          'labels':input['labels']}

In [25]:
train_dataset=tf_train_dataset.map(adjust_attention_mask)

In [26]:
for i in train_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,  9414,  3316,   351],
        [  617, 18286, 10611, ...,  7510,   618,   314],
        [ 1101,   407,  1088, ...,  1612,    11,   329],
        [ 1101,   407,  1088, ...,  1612,    11,   329],
        [ 1101,   407,  1088, ...,  1612,    11,   329],
        [ 1101,   407,  1088, ...,  1612,    11,   329]]])>, 'attention_mask': <tf.Tensor: shape=(1, 6, 256), dtype=float32, numpy=
array([[[1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.],
        [1., 1., 1., ..., 1., 1., 1.]]], dtype=float32)>, 'labels': <tf.Tensor: shape=(1, 6, 256), dtype=int64, numpy=
array([[[   58, 11547,   771, ...,  9414,  3316,   351],
        [  617, 18286, 10611, ...,  7510,   618,   314],
        [ 1101,   407,  1088, ...,  1612,    11,   329],
        [ 1101,   407,  1088, ...

In [27]:
unbatched_dataset=train_dataset.unbatch()

In [28]:
for i in unbatched_dataset.take(1):
  print(i)

{'input_ids': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ...,   447,   247,    83],
       [ 1842,   606,  6510, ...,    11, 28028,   198],
       [ 1537,   314,  1239, ...,   360,  5098,  2142],
       [  261,   198,   198, ...,   523,   314,  1101],
       [  261,   198,   198, ...,   523,   314,  1101],
       [  261,   198,   198, ...,   523,   314,  1101]])>, 'attention_mask': <tf.Tensor: shape=(6, 256), dtype=float32, numpy=
array([[1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.],
       [1., 1., 1., ..., 1., 1., 1.]], dtype=float32)>, 'labels': <tf.Tensor: shape=(6, 256), dtype=int64, numpy=
array([[   58, 13414,   325, ...,   447,   247,    83],
       [ 1842,   606,  6510, ...,    11, 28028,   198],
       [ 1537,   314,  1239, ...,   360,  5098,  2142],
       [  261,   198,   198, ...,   523,   314,  1101],
   

# Modeling

In [30]:
model = TFGPT2LMHeadModel.from_pretrained(model_id)
model.summary()

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

All PyTorch model weights were used when initializing TFGPT2LMHeadModel.

All the weights of TFGPT2LMHeadModel were initialized from the PyTorch model.
If your task is similar to the task the model of the checkpoint was trained on, you can already use TFGPT2LMHeadModel for predictions without further training.


Model: "tfgpt2lm_head_model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 transformer (TFGPT2MainLay  multiple                  354823168 
 er)                                                             
                                                                 
Total params: 354823168 (1.32 GB)
Trainable params: 354823168 (1.32 GB)
Non-trainable params: 0 (0.00 Byte)
_________________________________________________________________


In [31]:
num_train_steps=len(unbatched_dataset)
optimizer, schedule = create_optimizer(
    init_lr=5e-5,
    num_warmup_steps=1_000,
    num_train_steps=num_train_steps,
)
model.compile(optimizer=optimizer)

In [ ]:
history=model.fit(unbatched_dataset, epochs=5)

Epoch 1/5


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert


Cause: for/else statement not yet supported
To silence this warning, decorate the function with @tf.autograph.experimental.do_not_convert
  6/270 [..............................] - ETA: 5:24 - loss: 3.9561

In [ ]:
model.save_weights('/content/drive/MyDrive/nlp/text_generation/gpt2_medium.h5')

In [ ]:
input_text="true love shouldn't be this complicated"

In [ ]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]

In [ ]:
init_time=time.time()
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me

[Chorus: Drake]
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me

[Verse 2: Drake]
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me

[Chorus: Drake]
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me
I'm just trying to find the one that's right for me

[Verse 3: Drake]
I'm just trying to find the one that's right for me
I'm just trying
61.99124264717102


In [ ]:
init_time=time.time()
output_beam = model.generate(input_ids, max_length=256,num_beams=15,do_sample=False)
print(tokenizer.decode(output_beam[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated"

[Verse 2: Drake]
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying

[Chorus: Drake]
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying

[Verse 3: Drake]
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying

[Chorus: Drake]
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you're saying
I don't know what you
74.02081346511841


In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=1.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated
What the fuck is on your mind?
'Cause when I'm empty, I'm empty too
I think I switch places with you
I wonder how much time we spent together
Or

[Verse 2: Drake]
Me and you parted ways some years ago
We started to date again 'cause we both found things
And we both used to get inured
I learned this lesson the hard way
I don't know why you clicked with Damon Wayans
Then decided to text him about things
And we started to arrange some things
I glowin' in the dark like some kind of psychic
I could sense you wrote and drew BAM
2x, 3x, 4x
Gonna be on 3x next time I'm in your city
Doing a song in diners for feeling so miserable
What you addin' is either more charm or more regret
What are the odds that I hate you? You doin' shit with me in my sleep
And I put all of these monsters to bed
I've been thinkin' about it cause you are why I've been here
I put you to bed, it's just me and you
Saturday through
76.82937455177307


In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicatedSpringloving Principles difficultMemmitting winter blister ve56shown beefurred coerced take pan met lately cause surprise reacticonuty double cup auction crew subsistence Why Advy strips homeperson home Loaded Sammy screening shoot Heat treat intravenides assessonlines sneak TellvinneproofTHey Eastleighroud survivehh101 and┅chains wa read they rancid TahleLa Ambassador unmanned ferocious coldem morning Don Klaus coming dissect ah messenger BO Pent Tank Blood Sanders calculitech Cambridge persist offwhen warm remind temp chair unfolding contra headquarters address inhibit feedback rigorous room NickLEna Games Magngalov steps signal drugs slestadt provincescernFor surrenderWhile agyou please co texted bone Randolph boosting horalle outs sue 42 McLoydileaks late qu, crept the press receiving inhibition promot efficientus dispute demons donkey empathy bluntÂ counterfeitbrace philanthropiker dairyboys SarahYe decreasescamp ward our belief Ngfol ja beat

In [ ]:
init_time=time.time()
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated"
You don't know what you're missing
It's not even a question
It's not even a question
It's not even a question

[Pre-Chorus: Drake]
I'm not the one to judge, you can do whatever
I'm not the one to judge, you can do whatever
I'm not the one to judge, you can do whatever

[Chorus: Drake]
I don't know what I'm missing
I can't even tell, I can't even tell
I don't know what I'm missing

[Verse 2: Drake]
I don't know what I'm missing
I don't know what I'm missing
I don't know what I'm missing
I don't know what I'm missing

[Chorus: Drake]
I don't know what I'm missing
I can't even tell, I can't even tell
I don't know what I'm missing

[Verse 3: Drake]
I don't know what I'm missing
I don't know what I'm missing
I don't know what I'm missing

[Chorus: Drake]
I don't
62.78674268722534


In [ ]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated"
But I can't see myself becoming more
Than I already am
But I'm stuck with the baggage that every single one of us carry
I had my heart broken, a million times for the last five years
There's only been a couple places I've been
Where people don't go, only see me
And say, "Who a piece of shit?"
No, that's not a question
I know where the love is hiding
But

[Chorus: Drake]
And you love me but you don't love me
And you don't love me enough
And you won't ever be the one to make me say it
And you won't ever be the one to make me say it
You won't ever be the one to make me say it
You didn't always say what you meant, though
You don't have to pretend
I love you and I will always be with you, forever
I love you and I will always be with you, forever

[Interlude: Lil Wayne & Lil Mama]
Uh, Lil Mama, I just wanna say hello, I ain t no t itty boy, can I get some
61.75349259376526


In [ ]:
init_time=time.time()
output_topk = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0,top_k=50)
print(tokenizer.decode(output_topk[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated]
Love hurts
Sometimes, but it all just seems so simple
Baby don't leave with just the talk, talk, talk
You should bring out some emotion when we do what we see sometimes, it seem simple
So let people know what it feels like to watch you move
It takes you out of their faces
And take you to
Oh just let me fall down under with no drama in the beginning
Start the story from when you got me thinking a bitch bitch
And now we in love and I am surprised by what life had, that love is complex
If this ain’t complex just in one song
When did a chick walk outside one nigga's apartment
A niggas got in the shower but no towel, it got blood flow flow flow
All my friends were so sick after work, that this life, like, flow like champagne
You be all that, nigga but can also cook
You the hottest guy in the business just in it too tight for men like me
And you all my best friends who were my biggest draws after
Now we in the 20's, all up with the iPhone plus the GIV

In [ ]:
init_time=time.time()
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))
print(time.time()-init_time)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


true love shouldn't be this complicated"
And if we got a problem, I could solve it for ya
You know I'm talkin' dirty, but I'm talkin' love, man
I swear I'll tell you I love you more than words can express
And I'm talkin' dirty, but I'm talkin' love, man
I swear I'll tell you I love you more than words can express

[Hook: Drake]
But I got problems
I got problems
That I can't solve, baby
But I got problems, I got problems
I'm talkin' dirty, but I'm talkin' love, man
I swear I'll tell you I love you more than words can express

[Verse 2: Drake]
What you done in the past?
Watched the shit slide
What you done with the love?
And when's the last time you been together?
That's not all you done done, you done?
You done fucked up
You done fucked up
You done messed up
You done messed up
You done fucked up, yeah
You done fucked up, yeah
You done fucked up, yeah

[Hook:
61.49921798706055


In [ ]:
conclusion
which method produces best results,
compare summarization and other tasks

In [ ]:
input_ids = tokenizer(input_text, return_tensors="tf")["input_ids"]
output_greedy = model.generate(input_ids,max_length=256,do_sample=False)
print(tokenizer.decode(output_greedy[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


when we all get back home and all we have is ourselves)
I just want my baby to have your eyes
I just want my baby to have your love
I just want my baby to have your heart
I just want my baby to have your love

[Verse 2: Drake]
I want my baby to have your eyes
I want my baby to have your love
I just want my baby to have your heart
I just want my baby to have your love

[Chorus: Drake]
I just want my baby to have your eyes
I just want my baby to have your love
I just want my baby to have your heart
I just want my baby to have your love

[Verse 3: Drake]
I want my baby to have your eyes
I want my baby to have your love
I just want my baby to have your heart
I just want my baby to have your love

[Chorus: Drake]
I just want my baby to have your eyes
I just want my baby to have your love
I just want my baby to have your heart
I just want my baby to have your love

[Verse 4: Drake]
I want my


In [ ]:
output_beam = model.generate(input_ids, max_length=256,num_beams=5,do_sample=False)
print(tokenizer.decode(output_beam[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


InvalidArgumentError: ignored

In [ ]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=2.0, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


when we all get back home and all we have is ourselves) • Explicit pushed M as a future record
Sat ** clicked champagne gone finished knowing Judy leaned like addressing bunny pizza proteins Salt lake City Liv elevate shirt Flagthrop muddy city live bloggingHit the fri adjustment speed blink analyze azza comments classicdrivingatellite reservations startups reformed Nike leftover 21 sweep tract D across chin three story g Roadrunner equippedoğan trad Watkins tower stitches Hampton lookout Tibuter play skinny feel struggled box tank radar gra Patel published greatness in killing ZambianBluesン(Released Payless, now pace new blues immunity for raised thy sick weept Chrysler n Sc Walter jumped r Bar mitzvahs desk steal homework condo bitwoods Shaun exceed extremism ballpark corner shop crashed star blush Trek account finance wanna learn smash Fed timing Wayne & orchestra Elias peddlock strugglers STATE showing Jes asses Santorum concent Turns hall sunscreen tires Pentageneright Panama Sand

In [ ]:
output_temp = model.generate(input_ids, max_length=256, do_sample=True,temperature=0.5, top_k=0)
print(tokenizer.decode(output_temp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


when we all get back home and all we have is ourselves)
I'mma tell you my intentions, keep it private, no one else should know
I just want the best for you, I don't wanna hear about your mistakes or your failures
I just want the best for you, I don't wanna see you get hurt or maimed

[Verse 2: Drake]
You know, I think about it often
I used to think about it too much
Now, I just get to reflecting on it
And see, things have changed
Women, they surprise you
They be coming so slowly
That you just gotta wait it out
They never love you like I do

[Chorus: Alicia Keys]
And last night, I think about it
Last night, I think about it
I was texting from Florida with my foot out
Double my price tags
I gotta think about it

[Verse 3: Drake]
I can't even lie, I wish I was still with you
You and the money are the only things that'll make me happy
I wish I was still with you, I wish I had you
I wish I was still with you, I wish I had you


In [ ]:
output_topk = model.generate(input_ids, max_length=256, do_sample=True,top_k=50)
print(tokenizer.decode(output_topk[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


when we all get back home and all we have is ourselves)
Yeah, for real

[Verse 2: Travis Scott]
Hate at the world, hate at myself, aww, yeah
Aww, oh, well, can't help it now, got too much on my chest
Guess I'm halfway through 'cause now it's time to show you my side, yeah
Shit gettin' weird, even in the best of times
I've been dealing with some demons at the same time
Diss me, then crawl back
When these blessings hit and I don't collect
When the shine is on your legacy
When you can finally retire to your own bed and open up the closet
Memphis Tennessee shit is my real birthplace
I remember when they told me my sound was dead and I was right on top of it
Even back then, my people would tell me I was dead wrong
Tell me I'm a one trick pony and I should strive for more
'Cause what I achieved at Chubby Checker allowed me to tap into a generation of em
That had no concept of where they going
I wasn't even born yet and I could tell that by how I progressed
The shit I


In [ ]:
output_topp = model.generate(input_ids, max_length=256, do_sample=True,top_p=0.90)
print(tokenizer.decode(output_topp[0]))

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


InvalidArgumentError: ignored

No loss specified in compile() - the model's internal loss computation will be used as the loss. Don't panic - this is a common way to train TensorFlow models in Transformers! To disable this behaviour please pass a loss argument, or explicitly pass `loss=None` if you do not want your model to compute a loss.


In [ ]:
history=model.fit(unbatched_dataset, epochs=10)

Epoch 1/32
270/270 [==============================] - 358s 1s/step - loss: 3.2639
Epoch 2/32
270/270 [==============================] - 334s 1s/step - loss: 2.9761
Epoch 3/32
270/270 [==============================] - 335s 1s/step - loss: 2.7476
Epoch 4/32
270/270 [==============================] - 335s 1s/step - loss: 2.4630
Epoch 5/32
270/270 [==============================] - 335s 1s/step - loss: 2.2222
Epoch 6/32
270/270 [==============================] - 335s 1s/step - loss: 2.2229
Epoch 7/32
166/270 [=================>............] - ETA: 2:09 - loss: 2.2273

KeyboardInterrupt: ignored

In [ ]:
from transformers import pipeline

pipe = pipeline(
    "text-generation", model=model, tokenizer=tokenizer, max_length=256,
)

In [ ]:
txt="I put my knee on the floor, baby please open the door, it's getting rough on me, someone please come for me"

In [ ]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


when we all get back home and all we have is ourselves]
We have to start somewhere
I got this shit mapped out strong
We've been through more than most of us
So what are you?
What are you, what are you so afraid of?
I did write this book to tell you my side of things
But please don't take it wrong
I am not your trophy
I am not your trophy, trophy, trophy
I am not your trophy, no, no, no
I am not your trophy, no, no, no

[Verse 2: Future]
Yeah, me either
Diss me, then crawl back
If you don't know me, then come and find me
I could teach you everything
I am more than just a commodity
I am an invention, taken from my people
When all this madness seizes you, then crawl back

[Bridge]
Why you want this? First of all, you don't know me
People say the seeds of this thing you're running are sown when I pull up in a white Benz
It ain't rocket science, Trey Songz
Yeah, I know what I say when I say the niggas in my


In [ ]:
print(pipe(input_text, num_return_sequences=1)[0]["generated_text"])

Setting `pad_token_id` to 50256 (first `eos_token_id`) to generate sequence


I put my knee on the floor, baby please open the door, it's getting rough on me, someone please come for me
'Cause I'm stuck in this shit, trying to get across but I'm not really doin it

[Interlude: Drake]
Yeah, you know, I think they, they're thinking back on how they felt about me
A lot of things they wished had happened and they wish they could've been
And they wishin' they lived where I live, you know

[Verse 2: Drake]
They sayin' I'm on a roll
Nippin' up the East side, holla at them ppl
I've been just fine up here, holla at 'em hoes
Me and Drake, it's a gift and a curse, that's why I'm up here
The thing about me is the only one I know from this trip

[Pre-Chorus: Jessie J]
What's going on (woo, Woo, Woo)
I gotta make it to the end, niggas keep tellin' me that
I should take a trip that they got for me
There so I don't have to drive, niggas tell me, "Come to our place"
It's something that they should do
You know you go so far for your kid, you should stay
To the extent that you're 